In [ ]:
import numpy as np
import pandas as pd
from os.path import join, exists
from brainpipe.system import study

st = study('respi')
PATH_CYC = join(st.path,'cycles_by_cond/')
PATH_DATA = join(st.path,'database/')
subjects = ['MICP','SEMC','PIRJ','LEFC','FERJ','VACJ']
sessions = ['E1', 'E2']
window = 2. #length of the signal to take in seconds
sf = 512
bsl = 0.1

def create_trials(X,triggs,sf,window,bsl):
    npts, n_trig = X.shape[1], len(triggs)
    n_sec = npts/sf
    timings = np.arange(0,n_sec,n_sec/npts)
    data = np.array([])
    for tr in range(n_trig):
        sel = (timings > (triggs[tr]-bsl)) & (timings <= (triggs[tr]+window))
        npts_sel = round((window+bsl)*512)
        x = X[:,sel][np.newaxis]
        if sum(sel) != npts_sel:
            print('different sel size',sum(sel),'should be',npts_sel)
            x = x[:,:,:npts_sel]
        data = np.vstack((data,x)) if np.size(data) else x
        
    return data.swapaxes(0,1).swapaxes(1,2)

for su in subjects:
    filename = PATH_DATA+'Odor_No_Odor/{}_E_odor_phys.npz'.format(su)
    if not exists(filename):
        data_od, data_no_od = np.array([]), np.array([])
        for sess in sessions:
            #load all cycles information for all subjects and E sessions
            cycles = pd.read_excel(PATH_CYC+'All_cycles_features_E.xls','{}_{}'.format(su,sess))
            inspis = cycles['insp_time']
            #filter cycles to take
            t_od = inspis[np.load(PATH_CYC+'{}_{}_odorall.npy'.format(su,sess))==1.].values
            t_no_od = inspis[np.load(PATH_CYC+'{}_{}_no_odor.npy'.format(su,sess))==1.].values
            #load data
            mat = np.load(PATH_DATA+'{}_{}_sigs_bipo3_phys.npz'.format(su,sess))
            data = mat['x'][:,:,0] #remove last dimension (1)
            new_X_od = create_trials(data,t_od,sf,window,bsl)
            if su == 'VACJ' and sess=='E2':
                t_no_od = t_no_od[16:]
            new_X_no_od = create_trials(data,t_no_od,sf,window,bsl)
            #concatenate E1 E2 on the trial dimension
            data_od = np.concatenate((data_od,new_X_od),axis=2) if np.size(data_od) else new_X_od
            data_no_od = np.concatenate((data_no_od,new_X_od),axis=2) if np.size(data_no_od) else new_X_no_od
        dic = dict(zip((k for k in mat), (mat[k] for k in mat)))
        dic['x'] = data_od
        np.savez(filename,**dic)
        dic['x'] = data_no_od
        np.savez(filename.replace('odor_phys.npz','no_odor_phys.npz'),**dic)
    else:
        print(filename,'already computed')

### Downsampling data to 100Hz & normalize

In [ ]:
import numpy as np
from os.path import join, exists
from itertools import product
from brainpipe.system import study
from scipy.signal import decimate
from mne.baseline import rescale

st = study('respi')
PATH_DATA = join(st.path,'database/Odor_No_Odor/')
subjects = ['PIRJ']#['CHAF','MICP','SEMC','PIRJ','LEFC','FERJ','VACJ']
conds = ['odor','no_odor']
sf, new_sf = 512., 51.2
bsl, win = -0.1, 2.
# baseline =

#Downsampling to 100Hz data
for su, cond in product(subjects,conds):
    filename = PATH_DATA+'{}_E_{}_phys.npz'.format(su,cond)
    mat = np.load(filename)
    data = mat['x']
    ds_mat = decimate(data,int(sf/new_sf),axis=1).swapaxes(1,2) #time last Dim for rescale
    npts = ds_mat.shape[2]
    time = np.arange(bsl,win,(win-bsl)/npts)
    norm_mat = rescale(ds_mat, time, baseline=[bsl,0],mode='zscore')
    print(su, cond,data.shape,'decimate',ds_mat.shape,'norm',norm_mat.shape)
    mat_new = dict(zip((k for k in mat),(mat[k] for k in mat)))
    mat_new['x'], mat_new['sf'], mat_new['time'] = norm_mat.swapaxes(1,2), new_sf, time
    np.savez(filename.replace('.npz','_ds.npz'), **mat_new)

### Plot ERPs with stats

In [ ]:
import numpy as np
from os.path import join, exists
from itertools import product
from brainpipe.system import study
from brainpipe.statistics import perm_pvalue2level
from brainpipe.visual import *
from scipy import stats
from numpy.random import permutation
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
from joblib import delayed, Parallel

st = study('respi')
PATH_DATA = join(st.path,'database/Odor_No_Odor/')
PATH_SAVE = join(st.path,'ERPS_Encoding/')
subjects = ['PIRJ'] #'CHAF','MICP','SEMC','LEFC','FERJ','VACJ',
conds = ['odor','no_odor']
n_perm = 1000
plot = True

for su in subjects:
    filename = PATH_DATA+'{}_E_{}_phys_ds.npz'
    filename_save = PATH_SAVE+'{}_E_{}_phys_ds.npz'.format(su,'Odor_Conds')
    if not exists(filename_save):
        MAT = np.load(filename.format(su,conds[0]))
        channels, labels, time, mat0 = MAT['channels'], MAT['Mai_RL'], MAT['time'], MAT['x']
        mat1 = np.load(filename.format(su,conds[1]))['x']

        n_elecs, n_pts = mat0.shape[0], mat0.shape[1]
        T_val_su, Tperm_su = np.array([]), np.array([])
        for elec in range(n_elecs):
            chan, label = channels[elec], labels[elec]
            X_elec = np.concatenate((mat0[elec],mat1[elec]),axis=1)
            T_perm_elec = np.array([])
            pvals, T_vals = [], []
            for t in range(n_pts):
                X0, X1 = mat0[elec,t,:], mat1[elec,t,:]
                #t-test by time point
                T, p = stats.ttest_ind(X0,X1)
                pvals.append(p), T_vals.append(T)

                #permutation t-test
                X = np.concatenate((X0,X1),axis=0)
                y = [0]*len(X0)+[1]*len(X1)
                T_perm_t = []
                for perm in range(n_perm):
                    y_perm = X[permutation(len(y))]
                    T_perm, _ = stats.ttest_ind(y_perm[:int(len(X)/2)],y_perm[int(len(X)/2):])
                    T_perm_t.append(T_perm)
                T_perm_elec = np.vstack((T_perm_elec,T_perm_t)) if np.size(T_perm_elec) else T_perm_t
            print(T_perm_elec.shape)
            th_0_001_perm = perm_pvalue2level(T_perm_elec.swapaxes(0,1),p=0.001,maxst=True)[0]
            th_0_01_perm = perm_pvalue2level(T_perm_elec.swapaxes(0,1),p=0.01,maxst=True)[0]
            th_0_05_perm = perm_pvalue2level(T_perm_elec.swapaxes(0,1),p=0.05,maxst=True)[0]

            #criteria to be significant
            underp = np.where(T_vals > th_0_05_perm)[0]
            pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
            signif = [True for k in pvsplit if len(k) >= 2]

            if plot == True and len(signif)>1:
                #Plot figure 
                fig = plt.figure(1,figsize=(7,7))
                title = 'ERP T-test for '+su+' '+str(chan)+' '+str(label)+' ('+str(elec)+')'
                fig.suptitle(title, fontsize=12)

                # Plot the POW + STATS
                plt.subplot(211)    
                BorderPlot(time, X_elec, y=y, kind='sem', alpha=0.2, color=['orange','blue'],linewidth=2, 
                           ncol=1, xlabel='Time (s)',ylabel = r'Power', legend=['Odor','No Odor'])
                rmaxis(plt.gca(), ['right', 'top'])
                addLines(plt.gca(), vLines=[0], vColor=['darkgray'], vWidth=[2])
                plt.legend(loc=0, handletextpad=0.1, frameon=False)
                plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

                # Plot DA for the POW
                plt.subplot(212)
                plt.plot(time,abs(np.asarray(T_vals)),'k-')
                plt.xlabel('Time (s)')
                plt.ylabel('T_values')
                rmaxis(plt.gca(), ['right', 'top'])
                addLines(plt.gca(), vLines=[0], vColor=['darkgray'], vWidth=[2])
                plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
                plt.plot(time, th_0_05_perm*np.ones(len(time)), '--', color='yellow', linewidth=2)
                plt.plot(time, th_0_01_perm*np.ones(len(time)), '--', color='orange', linewidth=2)
                plt.plot(time, th_0_001_perm*np.ones(len(time)), '--', color='r', linewidth=2)

                plot_name = PATH_SAVE+'{}_{}_{}_Odor_No_odor_({}).png'.format(su,chan,label,elec)
                plt.savefig(plot_name, dpi=300, bbox_inches='tight')
                plt.clf()
                plt.close()

            Tperm_su = np.concatenate((Tperm_su,T_perm_elec[np.newaxis]),axis=0) if np.size(Tperm_su) else T_perm_elec[np.newaxis]
            T_vals = np.asarray(T_vals)[np.newaxis]
            T_val_su = np.concatenate((T_val_su,T_vals),axis=0) if np.size(T_val_su) else T_vals
        dico = dict(zip((k for k in MAT),(MAT[k] for k in MAT)))
        dico['x_od'], dico['x_no_od'] = mat0, mat1
        dico['T_perm'], dico['T_vals'] = Tperm_su, T_val_su
        np.savez(filename_save,**dico)
    else:
        print(filename_save,'already computed')


### Save only electrodes that respond to odors

In [ ]:
time = np.arange(0,2,2/n_pts)
#Plot figure 
fig = plt.figure(1,figsize=(7,7))
title = 'ERP T-test for '+su+' '+str(chan)+' '+str(label)+' ('+str(elec)+')'
fig.suptitle(title, fontsize=12)

